In [ ]:
#обработка данных и формирование таблица для переезда пользователей между платформами

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

import warnings
warnings.filterwarnings('ignore')

In [2]:
url = URL(
    user="ABUHTAREVICH",
    password="/",
    account="/",
    warehouse="/",
    database="SEVEREX",
    schema="/",
    #role='xxxxx',
    #authenticator='https://xxxxx.okta.com',
)
engine = create_engine(url)
connection = engine.connect()

In [45]:
# show all tables

pd.set_option('display.max_rows', 200)

In [46]:
query_idfa = '''
select 
    idfa as device_id,
    min(date_part(epoch_second, INSTALL_TIME)) as created_at,
    'Imported Devices' as network,
    CAMPAIGN as campaign,
    AF_ADSET as adgroup,
    AF_AD as creative
from t_events
where APP_ID = 'id1594851756'
and idfa is not null
group by 1, 3, 4, 5, 6
'''

In [47]:
query_idfv = '''
select 
    idfv as device_id,
    min(date_part(epoch_second, INSTALL_TIME)) as created_at,
    'Imported Devices' as network,
    CAMPAIGN as campaign,
    AF_ADSET as adgroup,
    AF_AD as creative
from t_events
where APP_ID = 'id1594851756'
and idfv is not null
group by 1, 3, 4, 5, 6
'''

In [48]:
idfa = pd.read_sql(query_idfa, connection)

In [49]:
idfv = pd.read_sql(query_idfv, connection)

In [50]:
idfa.head()

,device_id,created_at,network,campaign,adgroup,creative
0,EAF5E265-F20C-4F69-A3BE-BFE92EF04E9E,1.639578e+09,Imported Devices,None,None,None
1,2AFA9688-F2E7-4558-A60C-3F3322E987DB,1.642021e+09,Imported Devices,None,None,None
2,AD811A8A-B2FE-4762-B828-28C73B7DB1B4,1.643344e+09,Imported Devices,None,None,None
3,45B97FDF-3985-42D7-A0B9-AFC7B2156CAD,NaN,Imported Devices,iOS - TT - Charger - US - CPI // 06.01.22,iOS - TT - Charger - US - CPI // 06.01.22,VD_CA_07_apple_compare_9x16_EN_KK.mp4_001
4,5A0352DF-8548-477E-A404-F7C45D72418D,1.645031e+09,Imported Devices,None,None,None


In [51]:
idfa.shape

(941002, 6)

In [52]:
idfa.dtypes

device_id      object
created_at    float64
network        object
campaign       object
adgroup        object
creative       object
dtype: object

In [53]:
idfa.created_at = idfa.created_at.fillna(0).astype('int')

In [54]:
idfa.created_at.value_counts()

0             930972
1643068800         5
1648380323         5
1643155200         5
1642464000         4
               ...  
1644471288         1
1642020699         1
1642039791         1
1643997979         1
1656449347         1
Name: created_at, Length: 6740, dtype: int64

In [55]:
idfa.to_csv('b4k5i66510xs_ios_idfa.csv', index=False) 

In [56]:
idfv.head()

,device_id,created_at,network,campaign,adgroup,creative
0,A4EEF1B9-1131-4E7E-A8D9-D45F3D1516DB,1641770539,Imported Devices,None,None,None
1,16050186-9995-4F76-8EDF-70F4B0F83874,1641773417,Imported Devices,None,None,None
2,F39B91F0-FE0B-4C3E-9214-1F309792E67A,1641772943,Imported Devices,Unity - iOS - Charger - US - CPI // 06.01.22,VD_CA_07_apple_compare_EN_KK (EC - 1.2),VD_CA_07_apple_compare_EN_KK (EC - 1.2)
3,B8C3A2B1-D86A-42AE-8D00-B3457E8E497D,1641772957,Imported Devices,Unity - iOS - Charger - US - CPI // 06.01.22,VD_CA_07_apple_compare_EN_KK (EC - 1.2),VD_CA_07_apple_compare_EN_KK (EC - 1.2)
4,9CC0CFD3-3471-4446-B747-22C51DCC3F78,1641778964,Imported Devices,None,None,None


In [57]:
idfv.dtypes

device_id     object
created_at     int64
network       object
campaign      object
adgroup       object
creative      object
dtype: object

In [58]:
idfv.query('created_at == 0')

,device_id,created_at,network,campaign,adgroup,creative


In [59]:
idfv.to_csv('b4k5i66510xs_ios_idfv.csv', index=False) 